In [2]:
# Se importa la libreria random para hacer uso de números aleatorios, y una libreria para guardar listas
import pygame
import math
import random
import heapq
from collections import namedtuple
from collections import defaultdict, deque

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class Nodo: # Clase nodo guarda el nombre de cada nodo en forma de string (1)
    def __init__(self, valor):
        self.valor = valor
        self.vecinos = []
        self.vecinos2 = []
    # Se crea un metodo para agregar vecinos al arbol generado utilizado en BFS y DFS
    def agregar_vecino(self, vecino):
        self.vecinos.append(vecino)
    def agregar_vecino2(self, vecino, peso):
        self.vecinos2.append((vecino,peso))
    # El nodo se guarda con el formato dado desde la función
    def __str__(self):
        return str(self.valor)

class Arista: # Clase arista guarda el nombre de cada arista en forma de string ( 1 -> 2 )
    def __init__(self, origen, destino, peso):
        self.origen = origen
        self.destino = destino
        self.origen2 = origen
        self.destino2 = destino
        self.peso = peso
    # La arista se guarda con el formato 1 -> 2
    def __str__(self):
        return f"{self.origen} -> {self.destino}"

class Grafo:
    def __init__(self,nombre_archivo):
        # Se inicializa el arreglo de nodos y aristas del grafo
        self.nodos = {}
        self.aristas = {}
        self.pesos = {}
        self.aristas2 = {}
        self.pesos2 = {}
        self.aristas3 = []
        self.nombre_archivo = nombre_archivo
    
    # Se guarda cada nodo a una lista de nodos
    def agregar_nodo(self, valor):
        if valor not in self.nodos:
            nodo = Nodo(valor)
            self.nodos[valor] = nodo
    # Se guarda cada arista a una lista de aristas
    def agregar_arista(self, valor_origen, valor_destino, peso):
        #if valor_origen not in self.nodos and valor_destino not in self.nodos:
        # Se verifica que el nodo de origen y destino esten en la lista de nodos
        if valor_origen not in self.nodos:
            self.agregar_nodo(valor_origen)
        if valor_destino not in self.nodos:
            self.agregar_nodo(valor_destino)

        origen = self.nodos[valor_origen]
        destino = self.nodos[valor_destino]
        origen2 = self.nodos[valor_origen]
        destino2 = self.nodos[valor_destino]
        # Se agrega el nodo vecino para BFS y DFS
        origen.agregar_vecino(destino)
        
        origen2.agregar_vecino2(destino2, peso)
        destino2.agregar_vecino2(origen2, peso)

        arista = Arista(valor_origen, valor_destino, peso)
        self.aristas2[(valor_origen, valor_destino)] = arista
        self.pesos2[(valor_origen, valor_destino)] = peso
        
        self.aristas3.append(arista)
        self.pesos2[(valor_origen, valor_destino)] = peso
        self.pesos2[(valor_destino, valor_origen)] = peso
        
        arista = Arista(valor_origen, valor_destino, peso)
        if valor_origen not in self.aristas:
            self.aristas[valor_origen] = []
        self.aristas[valor_origen].append((arista, peso))
        
    def __str__(self):
        """    
        # Se guardan los valores de los nodos y aristas con el formato:
        digraph G {
            // Nodos
            1;
            2;
            3;
            // Aristas
            1 -> 2;
            1 -> 3;
            2 -> 3;
        }
        """
        output = "Grafo generado\ndigraph G { \n\t// Nodos:\n"
        for nodo in self.nodos.values():
            output += "\t" + str(nodo.valor) + "\n"

        output += "\t// Aristas:\n"
        for origen, aristas in self.aristas.items():
            for arista, peso in aristas:
                #output += "({}) --{}--> ({})\n".format(origen, peso, arista.destino)
                output += "\t{} -> {} [label=\"{}\"];\n".format(origen, arista.destino, peso)
        # Se guarda la cadena de texto como un archivo gv
        with open(self.nombre_archivo, "w") as archivo:
            archivo.write(output+"}")
        return output+"}"    
        
    # Se define el metodo para el algoritmo BFS
    def bfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        cola = deque([self.nodos[valor_inicio]]) # Se define una "cola" con los nodos visitados
        padres = {self.nodos[valor_inicio]: None} # Se guardan los nodos padres de cada arista para obtener el arbol BFS
        while cola: # Se establece la condicion de paro del algoritmo cando no hay mas elementos en la cola
            nodo_actual = cola.popleft()
            if nodo_actual not in visitado: # Se establece si el nodo actual no se ha visitado
                #print(nodo_actual.valor, end=" ")
                visitado.add(nodo_actual) # Se marca el nodo como visitado
                for vecino in nodo_actual.vecinos:
                    if vecino not in visitado:
                        cola.append(vecino) # Se agrega el nodo vecino a la cola
                        padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        arbol_bfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for nodo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_bfs.setdefault(padre.valor, []).append(nodo.valor)
        return arbol_bfs
    
    # Se define el metodo para el algoritmo DFS
    def dfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        padres = {self.nodos[valor_inicio]: None}  # Se guardan los nodos padres de cada arista para obtener el arbol 
        def dfs_recursivo(nodo_actual): # Se define una funcion con el algoritmo de forma iterativa
            #print(nodo_actual.valor, end=" ")
            visitado.add(nodo_actual) # Se marca el nodo como visitado
            for vecino in nodo_actual.vecinos:
                if vecino not in visitado:
                    dfs_recursivo(vecino)
                    padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        dfs_recursivo(self.nodos[valor_inicio]) # Se llama al algoritmo iterativo
        arbol_dfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for hijo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_dfs.setdefault(padre, []).append(hijo)
        return arbol_dfs
    
    def Dijkstra(self, valor_inicio):
        # Se inicializan todas las distancias a todos los nodos en infinito (nodos desconectados)
        distancias = {valor: float('inf') for valor in self.nodos}
        distancias[valor_inicio] = 0 # Se inicializa en cero la distancia desde el primer nodo
        predecesor = {valor: None for valor in self.nodos} # Se obtiene el siguiente nodo conectado al nodo actual
        visitados = set()
        while len(visitados) < len(self.nodos): # Se itera hasta que se recorren todos los nodos
            nodo_actual = None
            distancia_minima = float('inf')
            for valor, distancia in distancias.items():
                if valor not in visitados and distancia < distancia_minima:
                    nodo_actual = valor # Se establece el nodo actual
                    distancia_minima = distancia # Se establece la distancia al nodo actual
            if nodo_actual is None:
                break
            visitados.add(nodo_actual) # Se marcan los nodos visitados
            if nodo_actual in self.aristas:
                for arista, peso in self.aristas[nodo_actual]:
                    nueva_distancia = distancias[nodo_actual] + peso # Se acumulan los pesos
                    if nueva_distancia < distancias[arista.destino]:
                        distancias[arista.destino] = nueva_distancia
                        predecesor[arista.destino] = nodo_actual
        return distancias, predecesor
    
    def find(self, parent, i):
        if parent[i] == i:
            return i
        else:
            return self.find(parent, parent[i])

    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1
            
    def is_connected(self):
        # Verificar si el grafo está conectado utilizando BFS o DFS
        if not self.nodos:
            return True
        start = next(iter(self.nodos))
        visited = set()
        stack = [start]
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                for vecino in self.nodos[node].vecinos:
                    if vecino.valor not in visited:
                        stack.append(vecino.valor)
        return len(visited) == len(self.nodos)
            
    def kruskalD(self):
        # Lista para almacenar las aristas del MST
        mst = []
        # Ordenar las aristas por peso
        aristas_ordenadas = sorted(self.pesos2.items(), key=lambda item: item[1])
        parent = {}
        rank = {}
        # Inicializar los conjuntos disjuntos
        for nodo in self.nodos:
            parent[nodo] = nodo
            rank[nodo] = 0
        for (origen, destino), peso in aristas_ordenadas:
            u = origen
            v = destino
            uroot = self.find(parent, u)
            vroot = self.find(parent, v)
            if uroot != vroot:
                mst.append((self.aristas2[(origen, destino)], peso))
                self.union(parent, rank, uroot, vroot)
        return mst
    
    def kruskalI(self):
        # Ordenar las aristas por peso en orden descendente
        aristas_ordenadas = sorted(self.pesos2.items(), key=lambda item: item[1], reverse=True)
        # Eliminar aristas en orden descendente de peso, si no desconectan el grafo
        for (origen, destino), peso in aristas_ordenadas:
            # Eliminar temporalmente la arista
            self.nodos[origen].vecinos = [vecino for vecino in self.nodos[origen].vecinos if vecino.valor != destino]
            if not self.is_connected():
                # Si el grafo se desconecta, volver a añadir la arista
                self.nodos[origen].agregar_vecino(self.nodos[destino])
            else:
                # Eliminar la arista permanentemente
                del self.aristas2[(origen, destino)]
                del self.pesos2[(origen, destino)]
        # El MST estará compuesto por las aristas restantes en el grafo
        mst = [(arista, peso) for (origen, destino), arista in self.aristas2.items() for (key, peso) in self.pesos2.items() if key == (origen, destino)]
        return mst
    
    def prim(self, inicio):
        # Verificar si el nodo inicial existe en el grafo
        if inicio not in self.nodos:
            return None
        # Lista para almacenar las aristas del MST
        mst = []
        # Priority queue para seleccionar la arista de menor peso
        aristas_heap = []
        # Conjunto de nodos visitados
        visitados = set()
        # Función para agregar las aristas de un nodo a la priority queue
        def agregar_aristas(nodo):
            for vecino, peso in self.nodos[nodo].vecinos2:
                if vecino.valor not in visitados:
                    heapq.heappush(aristas_heap, (peso, nodo, vecino.valor))
        # Comenzar desde el nodo inicial
        visitados.add(inicio)
        agregar_aristas(inicio)
        while aristas_heap:
            peso, origen, destino = heapq.heappop(aristas_heap)
            if destino not in visitados:
                visitados.add(destino)
                mst.append((origen, destino, peso))
                agregar_aristas(destino)
        return mst

    def to_adjacency_list(self):
        adj_list = {}
        for nodo in self.nodos.values():
            if nodo.valor not in adj_list:
                adj_list[nodo.valor] = []
            for vecino in nodo.vecinos:
                adj_list[nodo.valor].append(vecino.valor)
                if vecino.valor not in adj_list:
                    adj_list[vecino.valor] = []
                if nodo.valor not in adj_list[vecino.valor]:
                    adj_list[vecino.valor].append(nodo.valor)
        return adj_list
    
class QuadTree:
    def __init__(self, boundary, n=1):
        self.boundary = boundary  # Rect bounds
        self.capacity = n  # Capacity
        self.points = []  # Points in this quad
        self.divided = False  # Is it divided?

    def subdivide(self):
        x, y, w, h = self.boundary
        self.nw = QuadTree((x, y, w / 2, h / 2), self.capacity)
        self.ne = QuadTree((x + w / 2, y, w / 2, h / 2), self.capacity)
        self.sw = QuadTree((x, y + h / 2, w / 2, h / 2), self.capacity)
        self.se = QuadTree((x + w / 2, y + h / 2, w / 2, h / 2), self.capacity)
        self.divided = True

    def insert(self, point):
        if not self.contains(self.boundary, point):
            return False

        if len(self.points) < self.capacity:
            self.points.append(point)
            return True
        else:
            if not self.divided:
                self.subdivide()

            if self.nw.insert(point):
                return True
            elif self.ne.insert(point):
                return True
            elif self.sw.insert(point):
                return True
            elif self.se.insert(point):
                return True

        return False

    @staticmethod
    def contains(boundary, point):
        x, y, w, h = boundary
        return x <= point.x < x + w and y <= point.y < y + h

    def query(self, boundary):
        found = []
        if not self.intersects(self.boundary, boundary):
            return found
        else:
            for p in self.points:
                if self.contains(boundary, p):
                    found.append(p)
            if self.divided:
                found.extend(self.nw.query(boundary))
                found.extend(self.ne.query(boundary))
                found.extend(self.sw.query(boundary))
                found.extend(self.se.query(boundary))
        return found

    @staticmethod
    def intersects(a, b):
        ax, ay, aw, ah = a
        bx, by, bw, bh = b
        return not (bx > ax + aw or bx + bw < ax or by > ay + ah or by + bh < ay)


In [4]:
def escribir_Dijkstra(grafo,n,nombre_archivo):
    print("Arbol de pesos minimos desde el nodo " + str(n) + "\ndigraph G_nodo_origen_" + str(n) + " {")
    distancias, predecesor = grafo.Dijkstra(n) # Se obtienen los nodos conectados al origen n, y su distancia
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G_nodo_origen_" + str(n) + " {\n")
        for nodo, distancia in distancias.items():
            if distancia == float('inf') or distancia == 0: # Se eliminan los nodos desconectados del arbol
                pass
            else:
                #print("Distancia desde el nodo {} hasta {}: {}".format(n, nodo, distancia))
                archivo.write("\t{} -> nodo_{}({});\n".format(n, nodo, distancia))
                print("\t{} -> nodo_{}({});".format(n, nodo, distancia))
        archivo.write("}")
    print("}")

In [5]:
def escribir_KruskalD(grafo,nombre_archivo):
    print("Arbol de expansion minima Kruskal directo \ndigraph G {")
    mst = grafo.kruskalD()
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for arista, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(arista.origen, arista.destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(arista.origen, arista.destino, peso))
        archivo.write("}")
    print("}")

In [6]:
def escribir_KruskalI(grafo,nombre_archivo):
    print("Arbol de expansion minima Kruskal inverso \ndigraph G {")
    mst = grafo.kruskalD()
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for arista, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(arista.origen, arista.destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(arista.origen, arista.destino, peso))
        archivo.write("}")
    print("}")

In [7]:
def escribir_Prim(grafo,nombre_archivo):
    print("Arbol de expansion minima Prim \ndigraph G {")
    mst = grafo.prim(1)
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for origen, destino, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(origen, destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(origen, destino, peso))
        archivo.write("}")
    print("}")

In [8]:
def randomErdos(n,m,p,w):
    nombre_archivo = "Erdos_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    # se conectan m nodos de forma aleatoria. (n>m)
    for i in range(m):
        u = random.randint(0,n-1)
        v = random.randint(0,n-1)
        if u != v: # Se omiten las conexiones hacia un mismo nodo
            g.agregar_arista((u),(v),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [9]:
def Malla(n,m,p,w):
    nombre_archivo = "Malla_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n*m nodos al grafo
    for i in range(n*m):
        g.agregar_nodo((i+1))
    # Se conectan los nodos de la malla i,j+1 y i+1,j
    for i in range(n):
        for j in range(m):
            # La malla se recorre de la siguiente forma:
            """
            1  2  3
            4  5  6
            7  8  9
            """
            u = i*n+j+1
            v1 = i*n+j+2
            v2 = (i+1)*n+j+1
            # Se descartan todas las posibles combinaciones cuyos nodos superen el valor total de nodos en el grafo
            if v1 > n*m or v2 > n*m:
                v2 = u
            if u != v1: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v1),random.randint(1,w))
            if u != v2: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v2),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [10]:
def Geografico(n,r,p,w):
    nombre_archivo = "Geografico_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    x = []; y = []
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
        # se define una posición aleatria del nodo en el espacio
        x.append(random.randint(0,3*r))
        y.append(random.randint(0,3*r))
    for i in range(n):
        for j in range(n):
            dist = (x[i]-x[j])**2+(y[i]-y[j])**2 # Se calcula la distancia entre los nodos del grafo
            # Se conectan los nodos que se encuentran dentro del rango de distancia dado
            if dist < r and i != j: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [11]:
def Gilbert(n,p,x,w):
    nombre_archivo = "Gilbert_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    for i in range(n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        for j in range(n):
            # Se conectan 2 nodos de forma aleatoria con una probabilidad p
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if random.random() <= p and i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j),random.randint(1,w))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(x)
    arbol_dfs = g.dfs(x)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,x,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [12]:
def Barabasi(n,m,p,w):
    nombre_archivo = "Barabasi_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan m nodos al grafo
    for i in range(1,m):
        g.agregar_nodo((i))
        # Se conectan los m nodos agregados
        for j in range(1,i):
            g.agregar_arista((i),(j),random.randint(1,w))
    # Se determina la preferencia de conexión de los nodos añadidos
    preferencia = [nodo for nodo in g.nodos.keys() for _ in range(len(g.nodos))]
    for i in range(m, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo(i) # Se agrega un nuevo nodo
        conexiones = random.sample(preferencia, m) # Se conecta el nuevo nodo al grafo
        for nodo in conexiones:
            cond = nodo in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != nodo and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista(i, nodo, random.randint(1,w))
                preferencia.extend([nodo, i])
            lista.append(nodo) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [13]:
def Dorogovtsev(n,p,w):
    nombre_archivo = "Dorogovtsev_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan 3 nodos al grafo
    for i in range(3):
        g.agregar_nodo(i)
    # Se conectan los nodos agregados formando un triángulo
    g.agregar_arista(1, 2, random.randint(1,w))
    g.agregar_arista(2, 3, random.randint(1,w))
    g.agregar_arista(3, 1, random.randint(1,w))
    # Se agregan los n-3 nodos restantes
    for i in range(4, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo((i)) # Se agrega un nuevo nodo
        for k in range(4):
            j = random.choice(list(g.nodos.keys())) # Se escoge el nodo destino de forma aleatoria de las aristas existentes
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista((i), (j), random.randint(1,w))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [14]:
# Algoritmo de Fruchterman y Reingold con optimización de Barnes-Hut
def fruchterman_reingold_barnes_hut(graph, positions, k, t, max_iterations, win, a, b):
    WIDTH, HEIGHT = 1800, 1000
    def cool(temp):
        return temp - (temp / max_iterations)

    for _ in range(max_iterations):
        forces = {node: Point(0, 0) for node in graph}
        boundary = (0, 0, WIDTH, HEIGHT)
        quadtree = QuadTree(boundary)

        for v in graph:
            quadtree.insert(Point(positions[v][0], positions[v][1]))

        # Calcular fuerzas de repulsión
        for v in graph:
            p_v = Point(positions[v][0], positions[v][1])
            for u in graph:
                if u != v:
                    p_u = Point(positions[u][0], positions[u][1])
                    delta = (p_v.x - p_u.x, p_v.y - p_u.y)
                    dist = math.sqrt(delta[0] ** 2 + delta[1] ** 2)
                    if dist > 0:
                        repulsive_force = a*k ** 2 / dist
                        forces[v] = Point(forces[v].x + delta[0] / dist * repulsive_force,
                                          forces[v].y + delta[1] / dist * repulsive_force)

        # Calcular fuerzas de atracción
        for v in graph:
            for u in graph[v]:
                p_v = Point(positions[v][0], positions[v][1])
                p_u = Point(positions[u][0], positions[u][1])
                delta = (p_v.x - p_u.x, p_v.y - p_u.y)
                dist = math.sqrt(delta[0] ** 2 + delta[1] ** 2)
                if dist > 0:
                    attractive_force = b*(dist ** 2) / k
                    forces[v] = Point(forces[v].x - delta[0] / dist * attractive_force,
                                      forces[v].y - delta[1] / dist * attractive_force)

        # Actualizar posiciones
        for v in graph:
            positions[v] = (
                1*(positions[v][0] + forces[v].x * t),
                1*(positions[v][1] + forces[v].y * t)
            )

            # Limitar las posiciones dentro de la ventana
            positions[v] = (
                min(max(positions[v][0], 50), WIDTH - 50),
                min(max(positions[v][1], 50), HEIGHT - 50)
            )

        t = cool(t)
        
        # Dibujar el grafo en cada iteración
        draw_graph(win, graph, positions)
        pygame.time.delay(5)

    return positions

In [15]:
# Función para dibujar el grafo
def draw_graph(win, graph, positions):
    # Colores
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)
    win.fill(WHITE)
    
    # Dibujar aristas
    for node in graph:
        for neighbor in graph[node]:
            pygame.draw.line(win, BLACK, positions[node], positions[neighbor], 1)
    
    # Dibujar nodos
    for node, pos in positions.items():
        pygame.draw.circle(win, RED, (int(pos[0]), int(pos[1])), 5)
    
    pygame.display.update()

In [16]:
def Grafo_dibujo(grafo,t,max_iterations,Point,a,b):
    # Configuración de Pygame
    pygame.init()
    WIDTH, HEIGHT = 1920, 1080
    win = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Force-Directed Graph Visualization")

    # Colores
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)
    
    # Convertir el grafo a una lista de adyacencia
    graph = grafo.to_adjacency_list()
    print(graph)
    print(grafo)
    
    # Parámetros del algoritmo de fuerzas dirigidas
    area = WIDTH * HEIGHT
    k = math.sqrt(area / len(graph))  # Constante de equilibrio

    # Inicializar posiciones de los nodos aleatoriamente dentro de los límites de la ventana
    positions = {node: (random.uniform(50, WIDTH - 50), random.uniform(50, HEIGHT - 50)) for node in graph}

    # Calcular la disposición usando el algoritmo de Fruchterman y Reingold con optimización de Barnes-Hut
    positions = fruchterman_reingold_barnes_hut(graph, positions, k, t, max_iterations, win, a, b)

    # Bucle principal
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        draw_graph(win, graph, positions)

    pygame.quit()

In [17]:
# Crear el grafo y agregar nodos y aristas
grafo1 = randomErdos(100,100,15,25)
grafo2 = randomErdos(350,350,15,25)
grafo3 = Malla(10,10,50,25)
grafo4 = Malla(17,20,15,25)
grafo5 = Geografico(100,10,20,25)
grafo6 = Geografico(350,10,50,25)
grafo7 = Gilbert(100,0.1,12,50)
grafo8 = Gilbert(350,0.01,110,75)
grafo9 = Dorogovtsev(100,14,25)
grafo10 = Dorogovtsev(350,150,25)
grafo11 = Barabasi(100,10,20,25)
grafo12 = Barabasi(350,10,150,25)

In [35]:
# Erdos 100 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.8 # Fuerzas de repulsion y atraccion
t = 0.005  # Factor de tiempo para iteraciones
max_iterations = 500  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo1,t,max_iterations,Point,a,b)

{0: [20], 20: [0], 1: [66, 8, 19], 66: [1, 41, 91, 75, 76], 8: [1, 45, 81], 19: [1, 91, 60], 2: [87, 23, 91], 87: [2, 33, 77, 83, 99], 23: [2, 40, 54, 94], 91: [2, 19, 66, 78], 3: [68], 68: [3], 4: [89, 31, 78, 78, 95], 89: [4], 31: [4], 78: [4, 40, 33, 81, 99, 85, 91], 5: [98, 50], 98: [5, 13, 56, 24], 6: [61], 7: [], 45: [8, 32, 82], 81: [8, 78, 28, 82, 69], 9: [15, 62], 15: [9, 16], 10: [14], 11: [96, 47, 59], 96: [11, 40, 70], 47: [11], 12: [], 13: [98, 76], 14: [10, 25, 22], 25: [14, 21], 16: [15, 49, 44], 49: [16, 79, 93], 44: [16, 67, 94, 93], 17: [53, 88], 18: [70, 55], 70: [18, 90, 96], 60: [19, 39, 54], 21: [25, 37], 37: [21], 22: [14, 26], 24: [98], 26: [36, 22], 36: [26], 27: [32, 80], 28: [81], 29: [73], 30: [], 32: [45, 27, 48, 38, 40, 41], 48: [32, 71], 38: [32], 33: [87, 78], 34: [77], 35: [88], 88: [35, 82, 17, 50], 39: [63, 60], 63: [39, 97], 40: [51, 32, 96, 23, 78], 51: [40], 41: [32, 66], 42: [], 43: [], 67: [44, 94], 94: [44, 67, 23], 82: [45, 81, 88], 46: [], 79:

In [36]:
# Erdos 350 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.7 # Fuerzas de repulsion y atraccion
t = 0.005  # Factor de tiempo para iteraciones
max_iterations = 200  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo2,t,max_iterations,Point,a,b)

{0: [], 1: [208, 161, 239], 208: [1, 166, 179, 262], 161: [1, 21, 47, 151, 91, 213], 239: [1, 332], 2: [23], 3: [348, 74, 245, 311], 348: [3, 226, 247, 91], 4: [37, 133], 37: [4, 264], 5: [304, 222, 39, 334], 304: [5], 222: [5, 98, 330], 6: [70], 70: [6], 7: [129, 173, 204, 310], 8: [106, 136, 211, 50], 106: [8, 341, 101], 136: [8, 86, 135, 11, 200, 322], 211: [8], 9: [112, 288], 112: [9], 10: [226], 226: [10, 48, 348], 11: [91, 169, 194, 172, 230, 136, 321], 91: [11, 161, 344, 348], 169: [11, 102, 160], 194: [11, 165, 224], 172: [11, 164, 278], 230: [11], 12: [322], 322: [12, 254, 136], 13: [109, 182, 95, 266], 109: [13, 324, 251], 182: [13], 95: [13, 311, 158, 332], 14: [66, 142, 216], 66: [14], 15: [289, 232, 290, 16], 289: [15, 44], 232: [15, 196, 205], 290: [15, 153], 16: [15, 49], 17: [33], 18: [113], 113: [18, 332], 19: [29, 35, 86], 20: [319, 274], 319: [20], 21: [161, 43, 196], 43: [21, 347, 328, 326, 76], 22: [84, 27], 84: [22, 163, 234, 184], 23: [2, 2], 24: [286], 25: [69, 

In [37]:
# Malla 100 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 2.2 # Fuerzas de repulsion y atraccion
t = 0.01  # Factor de tiempo para iteraciones
max_iterations = 500  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo3,t,max_iterations,Point,a,b)

{1: [2, 11], 2: [1, 3, 12], 11: [1, 10, 12, 21], 3: [2, 4, 13], 12: [2, 11, 13, 22], 4: [3, 5, 14], 13: [3, 12, 14, 23], 5: [4, 6, 15], 14: [4, 13, 15, 24], 6: [5, 7, 16], 15: [5, 14, 16, 25], 7: [6, 8, 17], 16: [6, 15, 17, 26], 8: [7, 9, 18], 17: [7, 16, 18, 27], 9: [8, 10, 19], 18: [8, 17, 19, 28], 10: [9, 11, 20], 19: [9, 18, 20, 29], 20: [10, 19, 21, 30], 21: [11, 20, 22, 31], 22: [12, 21, 23, 32], 23: [13, 22, 24, 33], 24: [14, 23, 25, 34], 25: [15, 24, 26, 35], 26: [16, 25, 27, 36], 27: [17, 26, 28, 37], 28: [18, 27, 29, 38], 29: [19, 28, 30, 39], 30: [20, 29, 31, 40], 31: [21, 30, 32, 41], 32: [22, 31, 33, 42], 33: [23, 32, 34, 43], 34: [24, 33, 35, 44], 35: [25, 34, 36, 45], 36: [26, 35, 37, 46], 37: [27, 36, 38, 47], 38: [28, 37, 39, 48], 39: [29, 38, 40, 49], 40: [30, 39, 41, 50], 41: [31, 40, 42, 51], 42: [32, 41, 43, 52], 43: [33, 42, 44, 53], 44: [34, 43, 45, 54], 45: [35, 44, 46, 55], 46: [36, 45, 47, 56], 47: [37, 46, 48, 57], 48: [38, 47, 49, 58], 49: [39, 48, 50, 59], 

In [21]:
# Malla 350 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.7 # Fuerzas de repulsion y atraccion
t = 0.008  # Factor de tiempo para iteraciones
max_iterations = 1000  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo4,t,max_iterations,Point,a,b)

{1: [2, 18], 2: [1, 3, 19], 18: [1, 17, 19, 35, 19, 35], 3: [2, 4, 20], 19: [2, 18, 20, 36, 20, 36], 4: [3, 5, 21], 20: [3, 19, 21, 37, 21, 37], 5: [4, 6, 22], 21: [4, 20, 22, 38], 6: [5, 7, 23], 22: [5, 21, 23, 39], 7: [6, 8, 24], 23: [6, 22, 24, 40], 8: [7, 9, 25], 24: [7, 23, 25, 41], 9: [8, 10, 26], 25: [8, 24, 26, 42], 10: [9, 11, 27], 26: [9, 25, 27, 43], 11: [10, 12, 28], 27: [10, 26, 28, 44], 12: [11, 13, 29], 28: [11, 27, 29, 45], 13: [12, 14, 30], 29: [12, 28, 30, 46], 14: [13, 15, 31], 30: [13, 29, 31, 47], 15: [14, 16, 32], 31: [14, 30, 32, 48], 16: [15, 17, 33], 32: [15, 31, 33, 49], 17: [16, 18, 34], 33: [16, 32, 34, 50], 34: [17, 33, 35, 51], 35: [18, 34, 36, 52, 36, 52], 36: [19, 35, 37, 53, 37, 53], 37: [20, 36, 38, 54, 38, 54], 38: [21, 37, 39, 55], 39: [22, 38, 40, 56], 40: [23, 39, 41, 57], 41: [24, 40, 42, 58], 42: [25, 41, 43, 59], 43: [26, 42, 44, 60], 44: [27, 43, 45, 61], 45: [28, 44, 46, 62], 46: [29, 45, 47, 63], 47: [30, 46, 48, 64], 48: [31, 47, 49, 65], 49

In [22]:
# Geografico 100 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 3 # Fuerzas de repulsion y atraccion
t = 0.005  # Factor de tiempo para iteraciones
max_iterations = 300  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo5,t,max_iterations,Point,a,b)

{0: [45], 45: [0, 0, 57], 1: [27, 42, 83, 84], 27: [1, 16, 1, 16, 42, 79, 83, 85], 42: [1, 16, 27, 1, 16, 27, 79, 85], 83: [1, 27, 33, 56, 59, 68, 1, 27, 33, 56, 59, 68], 84: [1, 36, 1, 36], 2: [10, 44, 87, 93], 10: [2, 2, 44, 58, 87, 93], 44: [2, 10, 2, 10, 67, 87], 87: [2, 10, 44, 67, 78, 2, 10, 44, 67, 78, 95], 93: [2, 10, 2, 10], 3: [4, 20, 21, 23, 69], 4: [3, 3, 23, 69], 20: [3, 3, 69], 21: [3, 8, 14, 19, 3, 8, 14, 19], 23: [3, 4, 3, 4], 69: [3, 4, 20, 61, 3, 4, 20, 61], 5: [39, 70], 39: [5, 5, 70], 70: [5, 39, 5, 39], 6: [40], 40: [6, 6], 7: [62, 72, 90], 62: [7, 7, 72], 72: [7, 62, 7, 62, 74, 90], 90: [7, 35, 72, 74, 7, 35, 72, 74], 8: [14, 19, 21], 14: [8, 8, 19, 21], 19: [8, 14, 8, 14, 21], 9: [15, 17, 29, 32], 15: [9, 9, 29, 32], 17: [9, 9, 29, 31, 32], 29: [9, 15, 17, 9, 15, 17, 32, 57], 32: [9, 15, 17, 29, 9, 15, 17, 29], 58: [10, 10], 11: [52], 52: [11, 11, 71, 75, 92], 12: [25], 25: [12, 13, 12, 13, 50], 13: [25, 50], 50: [13, 25, 13, 25], 16: [27, 42, 77, 79, 85], 77: [1

In [23]:
# Geografico 350 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.7 # Fuerzas de repulsion y atraccion
t = 0.001  # Factor de tiempo para iteraciones
max_iterations = 500  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo6,t,max_iterations,Point,a,b)

{0: [92, 102, 141, 147, 157, 165, 181, 183, 211, 239, 249, 263, 276, 284, 327, 334, 348], 92: [0, 3, 0, 3, 102, 123, 141, 147, 157, 165, 181, 183, 211, 239, 263, 276, 327, 334, 348], 102: [0, 92, 0, 92, 141, 157, 165, 211, 263, 297, 327, 334, 348], 141: [0, 41, 48, 92, 93, 102, 111, 0, 41, 48, 92, 93, 102, 111, 157, 165, 181, 211, 263, 327, 348], 147: [0, 92, 0, 92, 157, 165, 181, 183, 211, 215, 239, 249, 276, 284, 334, 348], 157: [0, 92, 102, 141, 147, 0, 92, 102, 141, 147, 165, 181, 183, 211, 239, 263, 276, 284, 327, 334, 348], 165: [0, 41, 92, 102, 141, 147, 157, 0, 41, 92, 102, 141, 147, 157, 181, 211, 263, 327, 334, 348], 181: [0, 11, 41, 92, 141, 147, 157, 165, 171, 0, 11, 41, 92, 141, 147, 157, 165, 171, 211, 215, 239, 263, 284, 327, 348], 183: [0, 3, 66, 92, 94, 123, 133, 147, 157, 0, 3, 66, 92, 94, 123, 133, 147, 157, 211, 239, 249, 276, 334, 348], 211: [0, 92, 102, 141, 147, 157, 165, 181, 183, 0, 92, 102, 141, 147, 157, 165, 181, 183, 239, 263, 276, 284, 327, 334, 348], 239:

In [24]:
# Gilbert 100 nodos
# Parámetros del algoritmo de resortes
a, b = 0.09, 1.8 # Fuerzas de repulsion y atraccion
t = 0.003  # Factor de tiempo para iteraciones
max_iterations = 300  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo7,t,max_iterations,Point,a,b)

{0: [19, 26, 30, 49, 52, 61, 77, 84, 86, 92, 2, 29, 45, 58, 80, 99], 19: [0, 11, 12, 0, 20, 24, 37, 75, 80, 97, 22, 23, 25, 31, 34, 35, 38, 44, 48, 53, 57, 66, 70, 88], 26: [0, 10, 8, 9, 15, 39, 44, 46, 57, 64, 71, 73, 80, 86, 36, 37, 42, 53, 91, 92, 98, 99], 30: [0, 9, 10, 20, 21, 29, 48, 53, 63, 81, 94, 57, 74, 80, 90, 95], 49: [0, 3, 29, 35, 40, 54, 56, 57, 59, 73, 98, 52, 85, 93, 95], 52: [0, 23, 28, 32, 46, 47, 15, 20, 49, 57, 66, 69, 88, 96, 72, 74, 75, 76, 92], 61: [0, 5, 32, 44, 50, 58, 4, 15, 18, 38, 40, 43, 46, 55, 56, 65, 71, 85, 91, 73, 75, 79, 87, 88, 95, 96, 97], 77: [0, 23, 28, 29, 35, 47, 56, 59, 69, 5, 8, 20, 37, 39, 44, 60, 71, 76, 80, 81, 92], 84: [0, 6, 17, 33, 43, 50, 64, 69, 70, 76, 31, 35, 40, 60, 64, 87, 88, 97, 98], 86: [0, 9, 10, 26, 41, 48, 53, 58, 74, 78, 79, 7, 10, 11, 16, 21, 37, 59, 62, 71, 73, 93], 92: [0, 4, 12, 22, 28, 44, 53, 55, 72, 74, 77, 80, 91, 5, 18, 26, 44, 45, 52, 73, 83, 93, 96], 1: [58, 74, 83, 85, 22, 32, 88, 89, 90], 58: [1, 11, 17, 22, 24

In [26]:
# Gilbert 350 nodos
# Parámetros del algoritmo de resortes
a, b = 0.09, 1.8 # Fuerzas de repulsion y atraccion
t = 0.003  # Factor de tiempo para iteraciones
max_iterations = 300  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo8,t,max_iterations,Point,a,b)

{0: [12, 32, 125, 7, 18, 152, 279], 12: [0, 61, 211, 122, 288], 32: [0, 117, 247, 267, 64, 160, 220], 125: [0, 23, 60, 320, 152, 327], 1: [4, 33, 37, 151, 161, 171, 306, 66, 186, 317], 4: [1, 21, 101, 111, 290], 33: [1, 30, 156, 65, 194, 314], 37: [1, 6, 141, 275, 144, 310, 316], 151: [1, 18, 199, 228, 308, 152, 153, 181, 218, 230], 161: [1, 96, 148, 154, 70, 169, 180, 221, 281, 200, 301], 171: [1, 159, 165, 229, 212, 276, 318, 325], 306: [1, 82, 241, 170, 222, 268, 310, 346, 321], 2: [71, 82, 305, 19, 56, 275, 282], 71: [2, 23, 59, 142, 94], 82: [2, 23, 101, 156, 216, 306, 337, 112, 168, 264, 280, 345], 305: [2, 219, 278, 299, 338], 3: [204, 21, 105, 189, 290], 204: [3, 28, 94, 198, 72, 278, 228, 238], 21: [4, 3, 34, 98, 107, 165, 206, 145, 267, 319], 101: [4, 82, 56, 83, 88, 110, 274, 328, 114, 138, 157], 111: [4, 52, 48, 57, 181, 340, 164], 290: [4, 34, 195, 3, 209, 258, 311], 5: [41, 175, 9, 158, 308], 41: [5, 128, 226, 244, 246, 311, 168, 177, 191, 292], 175: [5, 74, 102, 141, 76,

In [27]:
# Dorogovtsev 100 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.7 # Fuerzas de repulsion y atraccion
t = 0.005  # Factor de tiempo para iteraciones
max_iterations = 300  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo9,t,max_iterations,Point,a,b)

{0: [4, 7, 9, 11, 14, 20, 22, 29, 54, 90], 1: [2, 3, 5, 6, 8, 10, 13, 15, 20, 21, 23, 31, 37, 41, 83, 89], 2: [1, 3, 6, 9, 11, 12, 13, 20, 23, 32, 55, 65, 72, 77, 91], 3: [2, 1, 5, 7, 12, 15, 16, 21, 29, 43, 53, 62, 99], 4: [0, 8, 10, 17, 21, 22, 27, 29, 49, 56], 5: [3, 1, 6, 7, 8, 10, 15, 47, 52, 71], 6: [1, 5, 2, 7, 9, 16, 18, 74, 81, 86], 7: [6, 0, 5, 3, 12, 17, 26, 28, 32, 38], 8: [1, 4, 5, 9, 12, 13, 17, 20, 36, 41, 43, 82], 9: [6, 8, 2, 0, 22, 25, 37, 52, 58, 63, 77, 95], 10: [4, 5, 1, 11, 18, 19, 22, 28, 38, 53, 75, 97], 11: [10, 0, 2, 13, 14, 24, 42, 53, 67, 77, 81], 12: [7, 8, 2, 3, 14, 17, 61, 66, 74], 13: [11, 2, 8, 1, 16, 19, 34, 36, 43, 47, 69, 73, 91], 14: [12, 0, 11, 27, 49, 91], 15: [1, 5, 3, 19, 23, 26, 31, 42, 50, 58, 68, 78, 83, 88, 96], 16: [13, 6, 3, 18, 19, 82], 17: [4, 8, 12, 7, 25, 38, 41, 65, 67, 68], 18: [6, 16, 10, 32, 36, 37, 40, 44, 45, 48, 93], 19: [15, 16, 10, 13, 29, 30, 33, 46, 57, 60, 79], 20: [0, 8, 2, 1, 24, 25, 30, 34, 35, 75], 21: [3, 1, 4, 40, 43,

In [28]:
# Dorogovtsev 350 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.7 # Fuerzas de repulsion y atraccion
t = 0.004  # Factor de tiempo para iteraciones
max_iterations = 100  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo10,t,max_iterations,Point,a,b)

{0: [4, 5, 10, 20, 22, 29, 30, 31, 40, 106, 130, 147, 164, 205, 210], 1: [2, 3, 4, 7, 8, 15, 19, 25, 36, 43, 64, 93, 100, 127, 191, 193, 248], 2: [1, 3, 6, 7, 8, 10, 14, 15, 21, 31, 40, 75, 144, 149, 166, 188, 215, 294], 3: [2, 1, 4, 5, 8, 9, 12, 20, 76, 80, 85, 89, 93, 148, 165, 178, 247, 339], 4: [1, 3, 0, 6, 10, 12, 14, 15, 35, 42, 55, 68, 78, 110, 131, 271], 5: [3, 0, 7, 9, 12, 14, 21, 24, 32, 35, 38, 42, 46, 47, 57, 85, 87, 89, 92, 101, 118, 122, 161, 186, 203, 241, 245], 6: [2, 4, 7, 11, 13, 18, 19, 21, 22, 26, 28, 29, 30, 42, 62, 69, 137, 138, 213, 280, 349], 7: [2, 6, 5, 1, 11, 18, 21, 23, 39, 53, 56, 61, 85, 143, 222, 232, 265, 266, 269, 309], 8: [3, 2, 1, 10, 11, 13, 25, 38, 44, 49, 53, 60, 62, 69, 74, 90, 173, 174, 232, 234, 242, 243], 9: [5, 3, 15, 16, 23, 45, 47, 65, 80, 224, 283], 10: [4, 0, 8, 2, 11, 13, 14, 18, 20, 43, 48, 54, 63, 73, 81, 91, 92, 96, 107, 110, 129, 157, 230, 285, 343], 11: [6, 7, 8, 10, 12, 17, 33, 37, 39, 40, 46, 51, 53, 58, 96, 98, 314, 335], 12: [3, 

In [29]:
# Barabasi 100 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.5 # Fuerzas de repulsion y atraccion
t = 0.003  # Factor de tiempo para iteraciones
max_iterations = 300  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo11,t,max_iterations,Point,a,b)

{1: [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 20, 21, 22, 24, 28, 30, 31, 32, 33, 34, 35, 37, 40, 41, 48, 54, 58, 69, 76, 80, 94, 97], 2: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16, 17, 18, 19, 23, 25, 26, 29, 33, 35, 36, 42, 43, 49, 52, 55, 57, 60, 64, 68, 71, 73, 77, 80, 86, 91, 96], 3: [1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 18, 24, 28, 29, 30, 33, 35, 38, 43, 49, 51, 54, 60, 75, 78, 79, 87, 88, 96, 97], 4: [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 18, 19, 20, 21, 27, 29, 40, 46, 52, 56, 59, 61, 64, 65, 81, 84, 88, 96], 5: [1, 2, 3, 4, 6, 7, 8, 9, 12, 13, 15, 16, 19, 21, 22, 23, 24, 28, 29, 30, 34, 38, 44, 46, 47, 48, 49, 50, 54, 56, 67, 69, 70, 71, 72, 73, 75, 81, 93], 6: [1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 16, 17, 22, 25, 26, 31, 33, 36, 37, 38, 41, 44, 45, 57, 58, 60, 63, 71, 73, 78, 79, 82, 84, 86, 87, 93, 95], 7: [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20, 22, 25, 27, 30, 31, 32, 33, 34, 36, 37, 40, 41, 43, 47, 50, 51, 52, 53, 56, 58, 61, 62, 63, 72, 77, 82, 8

In [30]:
# Barabasi 350 nodos
# Parámetros del algoritmo de resortes
a, b = 0.1, 1.7 # Fuerzas de repulsion y atraccion
t = 0.001  # Factor de tiempo para iteraciones
max_iterations = 300  # Número máximo de iteraciones
Point = namedtuple('Point', ['x', 'y']) # Quadtree para la optimización de Barnes-Hut
Grafo_dibujo(grafo12,t,max_iterations,Point,a,b)

{1: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 18, 19, 20, 21, 22, 23, 25, 26, 27, 31, 33, 35, 36, 39, 40, 43, 47, 49, 51, 52, 54, 56, 63, 70, 71, 72, 73, 74, 91, 98, 101, 115, 119, 120, 123, 126, 129, 132, 136, 138, 158, 198, 199, 206, 220, 225, 227, 253, 258, 266, 269, 277, 285, 286, 293, 304, 309, 312, 329], 2: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16, 19, 24, 26, 29, 44, 58, 59, 61, 64, 90, 92, 104, 112, 121, 128, 136, 142, 145, 167, 190, 204, 214, 215, 220, 239, 263, 290, 318, 326], 3: [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 23, 27, 28, 30, 33, 36, 39, 43, 49, 57, 64, 69, 75, 77, 80, 82, 88, 89, 92, 93, 109, 111, 112, 115, 122, 125, 126, 133, 152, 159, 177, 178, 185, 186, 191, 192, 219, 221, 241, 256, 272, 288, 319, 326, 327], 4: [1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 14, 18, 19, 20, 22, 25, 29, 40, 63, 67, 68, 71, 73, 75, 78, 85, 95, 100, 110, 122, 126, 131, 133, 135, 137, 140, 141, 147, 158, 167, 168, 169, 180, 181, 183, 184, 188, 192, 193, 195, 200, 206, 208, 221, 2